In [1]:
!pip install pyaxis
!pip install pandasql

# Part 1: basics

This part is about converting natural language to SQL. It involves:
- **Injecting key context about our dataset** into the system prompt so that the LLM can compose the query.
- Calling an LLM via the OpenAI API
- Parsing the LLM's output to isolate the SQL query

In [2]:
from rich import print
from openai import OpenAI
import pandas as pd
from pandasql import sqldf
from google.colab import userdata
api_key = userdata.get('OpenAI_API')

client = OpenAI(api_key=api_key)

In [3]:
# helper functions

def get_variable_name(df):
    for name, obj in globals().items():
        if obj is df:
            return name
    return None

def parse_query(query):
    if "```sql" in query:
        return query.replace("```sql", "").replace("```", "").strip()
    elif "```" in query:
        return query.replace("```", "").strip()
    else:
        return None

def get_query_from_prompt(prompt, data: pd.DataFrame,
                          N: int = 20):


    SYSTEM_PROMPT = f"""You are an SQL expert. Your job is to convert the natural language query into an SQL query for a dataset named {get_variable_name(data)} -- this should be the name you use in your query.

    Below, I will provide some descriptive statistics, as well as an overview of the first {N} rows of the dataset you are working with.

    Description:

    {data.describe().to_markdown()}

    Head (first {N} rows):

    {data.head(N).to_markdown()}
    """

    response = client.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[
            {"role": "system", "content": SYSTEM_PROMPT},
            {"role": "user", "content": prompt},
        ],
        temperature=0.1,
    )

    return response.choices[0].message.content

In [4]:
with open("/content/sample_data/california_housing_test.csv") as f:
    df = pd.read_csv(f)

In [5]:
df

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
0,-122.05,37.37,27.0,3885.0,661.0,1537.0,606.0,6.6085,344700.0
1,-118.30,34.26,43.0,1510.0,310.0,809.0,277.0,3.5990,176500.0
2,-117.81,33.78,27.0,3589.0,507.0,1484.0,495.0,5.7934,270500.0
3,-118.36,33.82,28.0,67.0,15.0,49.0,11.0,6.1359,330000.0
4,-119.67,36.33,19.0,1241.0,244.0,850.0,237.0,2.9375,81700.0
...,...,...,...,...,...,...,...,...,...
2995,-119.86,34.42,23.0,1450.0,642.0,1258.0,607.0,1.1790,225000.0
2996,-118.14,34.06,27.0,5257.0,1082.0,3496.0,1036.0,3.3906,237200.0
2997,-119.70,36.30,10.0,956.0,201.0,693.0,220.0,2.2895,62000.0
2998,-117.12,34.10,40.0,96.0,14.0,46.0,14.0,3.2708,162500.0


In [6]:
df.describe()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
count,3000.000000,3000.00000,3000.000000,3000.000000,3000.000000,3000.000000,3000.00000,3000.000000,3000.00000
mean,-119.589200,35.63539,28.845333,2599.578667,529.950667,1402.798667,489.91200,3.807272,205846.27500
std,1.994936,2.12967,12.555396,2155.593332,415.654368,1030.543012,365.42271,1.854512,113119.68747
min,-124.180000,32.56000,1.000000,6.000000,2.000000,5.000000,2.00000,0.499900,22500.00000
25%,-121.810000,33.93000,18.000000,1401.000000,291.000000,780.000000,273.00000,2.544000,121200.00000
50%,-118.485000,34.27000,29.000000,2106.000000,437.000000,1155.000000,409.50000,3.487150,177650.00000
75%,-118.020000,37.69000,37.000000,3129.000000,636.000000,1742.750000,597.25000,4.656475,263975.00000
max,-114.490000,41.92000,52.000000,30450.000000,5419.000000,11935.000000,4930.00000,15.000100,500001.00000


In [7]:
df.head(30).to_markdown()

'|    |   longitude |   latitude |   housing_median_age |   total_rooms |   total_bedrooms |   population |   households |   median_income |   median_house_value |\n|---:|------------:|-----------:|---------------------:|--------------:|-----------------:|-------------:|-------------:|----------------:|---------------------:|\n|  0 |     -122.05 |      37.37 |                   27 |          3885 |              661 |         1537 |          606 |          6.6085 |               344700 |\n|  1 |     -118.3  |      34.26 |                   43 |          1510 |              310 |          809 |          277 |          3.599  |               176500 |\n|  2 |     -117.81 |      33.78 |                   27 |          3589 |              507 |         1484 |          495 |          5.7934 |               270500 |\n|  3 |     -118.36 |      33.82 |                   28 |            67 |               15 |           49 |           11 |          6.1359 |               330000 |\n|  4 |     -119

In [8]:
test = """```sql
SELECT COUNT(*)
FROM dataset
WHERE median_income < 5;
```"""

res = parse_query(test)

print(res)

SELECT COUNT(*)
FROM dataset
WHERE median_income < 5;

In [9]:
query = "I want to see only the data where the median income is below 5?"

sql_query = get_query_from_prompt(query, df)
print(f"sql_query: {sql_query}")

parsed_query = parse_query(sql_query)
print(f"parsed_query: {parsed_query}")

run_query = lambda q: sqldf(q, globals())

result_df = run_query(parsed_query)

display(result_df)

sql_query: ```sql
SELECT *
FROM ___
WHERE median_income < 5;
```

parsed_query: SELECT *
FROM ___
WHERE median_income < 5;

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
0,-118.30,34.26,43.0,1510.0,310.0,809.0,277.0,3.5990,176500.0
1,-119.67,36.33,19.0,1241.0,244.0,850.0,237.0,2.9375,81700.0
2,-119.56,36.51,37.0,1018.0,213.0,663.0,204.0,1.6635,67000.0
3,-121.43,38.63,43.0,1009.0,225.0,604.0,218.0,1.6641,67000.0
4,-120.65,35.48,19.0,2310.0,471.0,1341.0,441.0,3.2250,166900.0
...,...,...,...,...,...,...,...,...,...
2383,-118.23,34.09,49.0,1638.0,456.0,1500.0,430.0,2.6923,150000.0
2384,-119.86,34.42,23.0,1450.0,642.0,1258.0,607.0,1.1790,225000.0
2385,-118.14,34.06,27.0,5257.0,1082.0,3496.0,1036.0,3.3906,237200.0
2386,-119.70,36.30,10.0,956.0,201.0,693.0,220.0,2.2895,62000.0


# Part 2: Makstat

Now we escalate onto working with an actual MAKSTAT dataset, including:

- POST request to fetch the data
- Grappling with the .px data format

In [10]:
import requests
import json

URL = "https://makstat.stat.gov.mk:443/PXWeb/api/v1/mk/MakStat/DelovniSubj/DemografijaNaPretprijatija/DelovniTekovniPodatoci/350_DelSub_Mk_02_ml.px"

query = {
  "query": [
    {
      "code": "Година",
      "selection": {
        "filter": "item",
        "values": [
          "2023"
        ]
      }
    },
    {
      "code": "Мерки",
      "selection": {
        "filter": "item",
        "values": [
          "1"
        ]
      }
    }
  ],
  "response": {
    "format": "px"
  }
}

response = requests.post(URL, json=query)

# Check if the request was successful
if response.status_code == 200:
    print("Request successful!")
else:
    print(f"Request failed with status code: {response.status_code}")

Request successful!

In [11]:
from pyaxis import pyaxis

with open("result.px", 'wb') as f:
    f.write(response.content)

result = pyaxis.parse('result.px', encoding='utf-8')#, 'ISO-8859-5')

Multilingual PX file


In [12]:
print(result['DATA'])

Дејност Година                  Мерки  \
0                              Вкупно   2023  Активни претпријатија   
1                              Вкупно   2023  Активни претпријатија   
2                              Вкупно   2023  Активни претпријатија   
3                              Вкупно   2023  Активни претпријатија   
4                              Вкупно   2023  Активни претпријатија   
..                                ...    ...                    ...   
635  O96 Други лични услужни дејности   2023  Активни претпријатија   
636  O96 Други лични услужни дејности   2023  Активни претпријатија   
637  O96 Други лични услужни дејности   2023  Активни претпријатија   
638  O96 Други лични услужни дејности   2023  Активни претпријатија   
639  O96 Други лични услужни дејности   2023  Активни претпријатија   

                  Големина   DATA  
0                   Вкупно  67482  
1              0 вработени   5405  
2          1 - 4 вработени  49906  
3          5 - 9 вработени   6400  
4    10 и повеќе вработени   5771  
..                     ...    ...  
635                 Вкупно   2060  
636            0 вработени     80  
637        1 - 4 вработени   1881  
638        5 - 9 вработени     74  
639  10 и повеќе вработени     25  

[640 rows x 5 columns]

In [13]:
makstat = pd.DataFrame(result['DATA'])

In [14]:
makstat

,Дејност,Година,Мерки,Големина,DATA
0,Вкупно,2023,Активни претпријатија,Вкупно,67482
1,Вкупно,2023,Активни претпријатија,0 вработени,5405
2,Вкупно,2023,Активни претпријатија,1 - 4 вработени,49906
3,Вкупно,2023,Активни претпријатија,5 - 9 вработени,6400
4,Вкупно,2023,Активни претпријатија,10 и повеќе вработени,5771
...,...,...,...,...,...
635,O96 Други лични услужни дејности,2023,Активни претпријатија,Вкупно,2060
636,O96 Други лични услужни дејности,2023,Активни претпријатија,0 вработени,80
637,O96 Други лични услужни дејности,2023,Активни претпријатија,1 - 4 вработени,1881
638,O96 Други лични услужни дејности,2023,Активни претпријатија,5 - 9 вработени,74


In [15]:
makstat.describe()

,Дејност,Година,Мерки,Големина,DATA
count,640,640,640,640,640
unique,128,1,1,5,310
top,Вкупно,2023,Активни претпријатија,Вкупно,2
freq,5,640,640,128,27


In [16]:
makstat.head(20)

,Дејност,Година,Мерки,Големина,DATA
0,Вкупно,2023,Активни претпријатија,Вкупно,67482
1,Вкупно,2023,Активни претпријатија,0 вработени,5405
2,Вкупно,2023,Активни претпријатија,1 - 4 вработени,49906
3,Вкупно,2023,Активни претпријатија,5 - 9 вработени,6400
4,Вкупно,2023,Активни претпријатија,10 и повеќе вработени,5771
5,Б Рударство и вадење на камен,2023,Активни претпријатија,Вкупно,191
6,Б Рударство и вадење на камен,2023,Активни претпријатија,0 вработени,25
7,Б Рударство и вадење на камен,2023,Активни претпријатија,1 - 4 вработени,91
8,Б Рударство и вадење на камен,2023,Активни претпријатија,5 - 9 вработени,33
9,Б Рударство и вадење на камен,2023,Активни претпријатија,10 и повеќе вработени,42


In [17]:
makstat.head(20).to_markdown()

'|    | Дејност                                                                     |   Година | Мерки                 | Големина              |   DATA |\n|---:|:----------------------------------------------------------------------------|---------:|:----------------------|:----------------------|-------:|\n|  0 | Вкупно                                                                      |     2023 | Активни претпријатија | Вкупно                |  67482 |\n|  1 | Вкупно                                                                      |     2023 | Активни претпријатија | 0 вработени           |   5405 |\n|  2 | Вкупно                                                                      |     2023 | Активни претпријатија | 1 - 4 вработени       |  49906 |\n|  3 | Вкупно                                                                      |     2023 | Активни претпријатија | 5 - 9 вработени       |   6400 |\n|  4 | Вкупно                                                              

In [18]:
query = "Колку вкупно активни претпријатија имало во 2023 година во преработувачката индустрија?"

sql_query = get_query_from_prompt(query, makstat)
parsed_query = parse_query(sql_query)
print(f"parsed_query: {parsed_query}")

run_query = lambda q: sqldf(q, globals())

result_df = run_query(parsed_query)

display(result_df)

parsed_query: SELECT DATA
FROM makstat
WHERE Дејност = 'В Преработувачка индустрија'
  AND Година = 2023
  AND Мерки = 'Активни претпријатија'
  AND Големина = 'Вкупно';

,DATA
0,7810


The answer is correct!

This example, of course, relies on the relevant key ("преработувачка индустрија") being present in the head() overview. This will not always be the case. Think about ways to show the LLM all the relevant categories that it can query.

# Part 3: which dataset?

A significant part of the challenge will be enabling the LLM to decide which dataset to query via the API with what parameters at first. More next time!